# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
import json
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
import joblib 
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


### Initialize workspace

In [2]:
ws = Workspace.from_config()

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-140966
aml-quickstarts-140966
southcentralus
6b4af8be-9931-443e-90f6-c4c34a1f9737


### Create an Azure ML experiment

In [3]:
# choose a name for experiment
experiment_name = 'heart-failure-experiment'

experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

## Dataset

### Overview
This task is to predict mortality caused by heart disease. 

Attribute information:

age - Age of the patient<br />
anemia - Decrease of red blood cells or hemoglobin (boolean)<br />
creatinine_phosphokinase - Level of the CPK enzyme in the blood (mcg/L)<br />
diabetes - If the patient has diabetes (boolean)<br />
ejection_fraction - Percentage of blood leaving the heart at each contraction (percentage)<br />
high_blood_pressure - If the patient has hypertension (boolean)<br />
platelets - Platelets in the blood (kiloplatelets/mL)<br />
serum_creatinine - Level of serum creatinine in the blood (mg/dL)<br />
serum_sodium - Level of serum sodium in the blood (mEq/L)<br />
sex - Woman or man (binary)<br />
smoking - If the patient smokes or not (boolean)<br />
time - Follow-up period (days)<br />
DEATH_EVENT - If the patient deceased during the follow-up period (boolean)<br />

In [7]:
amlcompute_cluster_name = "project-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Cluster exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Cluster exists.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
key = "heart-failure"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:
# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric" : 'accuracy',
    "enable_early_stopping": True,
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 10
}
automl_config = AutoMLConfig(
    task = "classification",
    compute_target = 'project-compute',
    training_data=dataset,
    label_column_name="DEATH_EVENT",
    debug_log = "automl_errors.log",
    **automl_settings
    )

In [16]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on project-compute with default configuration
Running on remote compute: project-compute
Parent Run ID: AutoML_fffb52ea-e670-4e3b-9ff4-3b6764017b23

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value impu

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_fffb52ea-e670-4e3b-9ff4-3b6764017b23',
 'target': 'project-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-03-20T17:00:06.021917Z',
 'endTimeUtc': '2021-03-20T17:17:38.769692Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'project-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-experiment","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-140966","workspace_name":"quick-starts-ws-140966","region":"southcentralus","compute_target":"project-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [25]:
best_auto_run, best_auto_model = remote_run.get_output()
print(best_auto_run)

ConfigException: ConfigException:
	Message: The model you attempted to retrieve requires 'xgboost' to be installed at '==0.90'. You have 'xgboost==1.3.3', please reinstall 'xgboost==0.90' (e.g. `pip install xgboost==0.90`) and rerun the previous command.
	InnerException: XGBoostError: [17:38:27] /mnt/xgboost/src/learner.cc:922: Check failed: header == serialisation_header_: 

  If you are loading a serialized model (like pickle in Python) generated by older
  XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version.  There's a simple script for helping
  the process. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for reference to the script, and more details about differences between saving model and
  serializing.


Stack trace:
  [bt] (0) /anaconda/envs/azureml_py36/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x7c) [0x7f6922d5d64c]
  [bt] (1) /anaconda/envs/azure

In [ ]:
#TODO: Save the best model
best_auto_model._final_estimator

In [ ]:
joblib.dump(best_auto_model, filename="./automl_best.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service